In [1]:
#import dependencies
import pandas as pd
import spotipy
from config import CLIENT_ID, CLIENT_SECRET
import requests
import json

In [3]:
# read in chart csv
data = pd.read_csv("Data/billboard_hot100_16-21.csv")
chart_df = pd.DataFrame(data)
chart_df.head()

#add new row in chart df 
chart_df["Spotify_Artist"] = ""
chart_df.head()

# trim down dataframe by dropping duplicate songs and reset index
unique_songs_df = chart_df.drop_duplicates(subset=["Title", "Artist"])
unique_songs_df.reset_index(inplace=True, drop=True)
unique_songs_df.tail(200)

,Date,Title,Artist,Rank,Is_New,Image,Peak_Posistion,Last_Position,Weeks,Spotify_Artist
2860,2021-02-20,Neighbors,Pooh Shiesty Featuring BIG30,51,True,NaN,51,0,1,
2861,2021-02-20,GNF (OKOKOK),Polo G,55,True,NaN,55,0,1,
2862,2021-02-20,Glad You Exist,Dan + Shay,63,True,NaN,63,0,1,
2863,2021-02-20,Provide,G-Eazy Featuring Chris Brown & Mark Morrison,64,True,NaN,64,0,1,
2864,2021-02-20,Box Of Churches,Pooh Shiesty Featuring 21 Savage,81,True,NaN,81,0,1,
...,...,...,...,...,...,...,...,...,...,...
3055,2021-06-19,Snowflakes,Tom MacDonald,71,True,NaN,71,0,1,
3056,2021-06-19,That's Facts,Lil Baby & Lil Durk,73,True,NaN,73,0,1,
3057,2021-06-19,Please,Lil Baby & Lil Durk,79,True,NaN,79,0,1,
3058,2021-06-19,Up The Side,"Lil Baby, Lil Durk & Young Thug",80,True,NaN,80,0,1,


In [4]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
token = auth_response_data['access_token']

sp = spotipy.Spotify(token)

In [5]:
#testing
#5s91S8hSrJXqYdhwHOEioV "You Can Have It All" - "Yo La Tengo"
artist= """Yo La Tengo"""
track= """You Can Have It All"""
track_info = sp.search(q='artist:' + artist + ' track:' + track, type='track')
#print(json.dumps(track_id, indent=4, sort_keys=True))
track_id = track_info['tracks']['items'][0]['id']
track_id

'5s91S8hSrJXqYdhwHOEioV'

In [6]:
#Iterates through chart and formats artist names for spotify api
for index, row in unique_songs_df.iterrows():
    no_feat = row["Artist"].split(" Featuring")[0]
    first_artist = no_feat.split(" &")[0]
    unique_songs_df.iat[index, 9] = first_artist

unique_songs_df.head()

,Date,Title,Artist,Rank,Is_New,Image,Peak_Posistion,Last_Position,Weeks,Spotify_Artist
0,2016-01-02,Hello,Adele,1,False,NaN,1,1,8,Adele
1,2016-01-02,Sorry,Justin Bieber,2,False,NaN,2,2,8,Justin Bieber
2,2016-01-02,Hotline Bling,Drake,3,False,NaN,2,3,20,Drake
3,2016-01-02,Love Yourself,Justin Bieber,4,False,NaN,4,5,5,Justin Bieber
4,2016-01-02,What Do You Mean?,Justin Bieber,5,False,NaN,1,4,16,Justin Bieber


In [16]:
# Creates songs data dict, iterates through chart and searches spotify api for spotify ID
songs_data = {"Artist": [], "Spotify_Artist": [], "Track": [], "Track_ID": []}
data_not_found = {"Artist": [], "Spotify_Artist": [], "Track": [], "Track_ID": []}
for index, row in unique_songs_df.iterrows():
    artist = row["Artist"]
    spotify_artist = row["Spotify_Artist"]
    track = row["Title"]
    try:
        track_info = sp.search(q='artist:' + spotify_artist + ' track:' + track, type='track')
        track_id = track_info['tracks']['items'][0]['id']
    except:
        track_id = "not found"
        data_not_found["Artist"].append(artist)
        data_not_found["Spotify_Artist"].append(spotify_artist)
        data_not_found["Track"].append(track)
        data_not_found["Track_ID"].append(track_id)
        #print(track + " by " + artist + " not found")

    songs_data["Artist"].append(artist)
    songs_data["Spotify_Artist"].append(spotify_artist)
    songs_data["Track"].append(track)
    songs_data["Track_ID"].append(track_id)


In [17]:
#creates dataframe for songs WITH AND WITHOUT track IDs
songs_df = pd.DataFrame(songs_data)
songs_df.head()

,Artist,Spotify_Artist,Track,Track_ID
0,Adele,Adele,Hello,62PaSfnXSMyLshYJrlTuL3
1,Justin Bieber,Justin Bieber,Sorry,402qo6bnZJwCN1RUenyNuM
2,Drake,Drake,Hotline Bling,0wwPcA6wtMf6HUMpIRdeP7
3,Justin Bieber,Justin Bieber,Love Yourself,3dNcOt7h5DuBgahMkOrL1m
4,Justin Bieber,Justin Bieber,What Do You Mean?,4Vx8DGMNlFt6q3yxVoV8qK


In [18]:
#creates dataframe for songs WITHOUT track IDs
no_data_df = pd.DataFrame(data_not_found)
len(no_data_df)

167

In [19]:
# Turn songs dataframe into csv
songs_df.to_csv("Data/spotify_ids.csv", index=False)
no_data_df.to_csv("Data/missing_ids.csv", index=False)